<center><h1>Exploring the Competitors</h1></center>

### Part 1. Get location data using Foursquare

[Foursquare Places API](https://location.foursquare.com/products/places-api/) is very usefule online application used by many developers & other application like Uber etc. In this project you can used it to retrieve informtion about the places present in the neighborhoods of Toronto. The API returns a JSON file and we need to turn that into a data-frame. Here I’ve chosen similar businesses(pet grooming) for each neighborhood within a radius of 2.5km.

You will need to create an account with Foursquare to access the API.  It is free to sign up, and you get $200 free credit.



In [2]:
import requests
import pandas as pd

In [3]:
toronto_DF = pd.read_csv('toronto_postcode.csv')

In [4]:
display(toronto_DF)

,Postalcode,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
...,...,...,...,...,...
98,M9N,43.706876,-79.518188,York,Weston
99,M9P,43.696319,-79.532242,Etobicoke,Westmount
100,M9R,43.688905,-79.554724,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,43.739416,-79.588437,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [5]:
# Set up your API key and other constants for Foursquare API
API_KEY = 'fsq3lzvo9DnZ5k8dEVe6kmkNP6md9kBghLGGrfqYoNrl2rU='  # Replace with your actual Foursquare API key
LIMIT = 20  # Maximum number of venues to return
radius = 2500  # Search radius in meters

# a function to loop through each neighbourhood in the postalcode.csv file, and search for places
def getNearbyVenues(names, postalcode, latitudes, longitudes, radius):
    venues_list = []

    for name, postalcode, lat, lng in zip(names, postalcode, latitudes, longitudes):
        print(f"Processing neighborhood: {name}")

        # Skip if latitude or longitude is missing
        if pd.isnull(lat) or pd.isnull(lng):
            print(f"Skipping {name} due to missing latitude or longitude.")
            continue

        # Create the API request URL and parameters
        # Change the params here to customize your search, https://docs.foursquare.com/developer/reference/place-search
        # The following example looks for pet groomers in the neighbourhood
        # I am searching with categories ID 11134 - pet grooming services
        url = 'https://api.foursquare.com/v3/places/search'
        params = {
            'll': f'{lat},{lng}',
            'radius': radius,
            'limit': LIMIT,
            'categories': 11071
        }

        # Set up headers with the API key
        headers = {
            "Accept": "application/json",
            "Authorization": API_KEY  
        }

        # Make the GET request
        response = requests.get(url, headers=headers, params=params)

        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to get data for {name}. Status code: {response.status_code}")
            print(f"Error message: {response.text}")
            continue

        results = response.json()

        # Parse the JSON response
        for venue in results.get('results', []):
            # Extract venue information
            venue_name = venue.get('name')
            venue_id = venue.get('fsq_id')
            venue_location = venue.get('geocodes', {}).get('main', {})
            venue_lat = venue_location.get('latitude')
            venue_lng = venue_location.get('longitude')
            venue_categories = venue.get('categories', [])
            venue_category = venue_categories[0]['name'] if venue_categories else 'Unknown'

            # Append to the list
            venues_list.append([
                name,
                postalcode,
                lat,
                lng,
                venue_name,
                venue_id,
                venue_lat,
                venue_lng,
                venue_category
            ])

    # Create a DataFrame from the list
    nearby_venues = pd.DataFrame(venues_list, columns=[
        'Neighborhood',
        'Postalcode',
        'Neighborhood Latitude',
        'Neighborhood Longitude',
        'Venue',
        'fsq_id',
        'Venue Latitude',
        'Venue Longitude',
        'Venue Category'
    ])

    return nearby_venues

# Example usage with your DataFrame 'toronto_DF'
# Ensure 'toronto_DF' has the columns 'Neighborhood', 'Latitude', 'Longitude'
toronto_venues = getNearbyVenues(
    names=toronto_DF['Neighbourhood'],
    postalcode=toronto_DF['Postalcode'],
    latitudes=toronto_DF['Latitude'],
    longitudes=toronto_DF['Longitude'],
    radius=radius
)

# Display the first few rows of the resulting DataFrame
toronto_venues.head()


Processing neighborhood: Malvern, Rouge
Processing neighborhood: Rouge Hill, Port Union, Highland Creek
Processing neighborhood: Guildwood, Morningside, West Hill
Processing neighborhood: Woburn
Processing neighborhood: Cedarbrae
Processing neighborhood: Scarborough Village
Processing neighborhood: Kennedy Park, Ionview, East Birchmount Park
Processing neighborhood: Golden Mile, Clairlea, Oakridge
Processing neighborhood: Cliffside, Cliffcrest, Scarborough Village West
Processing neighborhood: Birch Cliff, Cliffside West
Processing neighborhood: Dorset Park, Wexford Heights, Scarborough Town Centre
Processing neighborhood: Wexford, Maryvale
Processing neighborhood: Agincourt
Processing neighborhood: Clarks Corners, Tam O'Shanter, Sullivan
Processing neighborhood: Milliken, Agincourt North, Steeles East, L'Amoreaux East
Processing neighborhood: Steeles West, L'Amoreaux West
Processing neighborhood: Upper Rouge
Processing neighborhood: Hillcrest Village
Processing neighborhood: Fairview,

,Neighborhood,Postalcode,Neighborhood Latitude,Neighborhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",M1B,43.806686,-79.194353,Blue Ocean Nails & Spa,4bb7ab84314e95214481479d,43.801081,-79.199017,Nail Salon
1,"Malvern, Rouge",M1B,43.806686,-79.194353,Nails for you,4c7bfb4993ef236a8bf6b60f,43.806984,-79.222775,Nail Salon
2,"Malvern, Rouge",M1B,43.806686,-79.194353,Nails For You,e72e5651d5c643192ecc9296,43.806573,-79.222242,Nail Salon
3,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497,Jenny's Nail Studio,4c54567da724e21e73793bf6,43.784042,-79.158588,Nail Salon
4,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497,KN'S Nails & Spa,5d80e0dab6277c000866ba5e,43.783375,-79.169641,Nail Salon


In [8]:
# The total number of nail services in Toronto
toronto_venues['Venue'].nunique()

413

In [9]:
# Oberserving the number of competitors in each neighbourhood
toronto_venues.groupby('Neighborhood')['Venue'].count()

Neighborhood
Agincourt                                           6
Alderwood, Long Branch                              9
Bathurst Manor, Wilson Heights, Downsview North     9
Bayview Village                                    10
Bedford Park, Lawrence Manor East                  20
                                                   ..
Willowdale, Newtonbrook                            20
Woburn                                              8
Woodbine Heights                                   13
York Mills West                                    16
York Mills, Silver Hills                            6
Name: Venue, Length: 102, dtype: int64

In [10]:
sort_count = toronto_venues.groupby('Neighborhood')['Venue'].count()
sort_count.sort_values()

Neighborhood
Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens                                         3
Clairville, Humberwood, Woodbine Downs, West Humber, Kipling Heights, Rexdale, Elms, Tandridge, Old Rexdale     3
Downsview Northwest                                                                                             3
Guildwood, Morningside, West Hill                                                                               3
Malvern, Rouge                                                                                                  3
                                                                                                               ..
Enclave of M5E                                                                                                 20
Regent Park, Harbourfront                                                                                      20
Richmond, Adelaide, King                                                   

### Part 2. Interactive leaflet map using coordinate data.


In [11]:
import folium # map rendering library

In [12]:
# Target in M5V
target = "CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport"

# Filter venues in the target neighborhood
search_area_M5V = toronto_venues[toronto_venues['Neighborhood'].str.contains(target, na=False)]
latitude = toronto_DF[toronto_DF['Neighbourhood'].str.contains(target, na=False)]['Latitude'].iloc[0]
longitude = toronto_DF[toronto_DF['Neighbourhood'].str.contains(target, na=False)]['Longitude'].iloc[0]

# Display the filtered data
display(search_area_M5V)

,Neighborhood,Postalcode,Neighborhood Latitude,Neighborhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category
955,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,43.628947,-79.39442,Yeti Nails & Spa,4cc99fdd786e4688d2379509,43.647996,-79.396708,Nail Salon
956,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,43.628947,-79.39442,Pinky's Nails,4fe254576d86fad7b5763fc6,43.646036,-79.406679,Nail Salon
957,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,43.628947,-79.39442,Sparks,4d5596f3cff7721e788eabf5,43.641710,-79.414449,Nail Salon
958,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,43.628947,-79.39442,The Ten Spot,4aee01bef964a52034d121e3,43.646453,-79.407120,Nail Salon
959,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,43.628947,-79.39442,Polish'd Nail Bar,5077398de4b0eb9f2a41e7ec,43.648120,-79.383405,Nail Salon
960,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,43.628947,-79.39442,Rite Spa,59ac224c3c858d09e3563900,43.637712,-79.393012,Nail Salon
961,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,43.628947,-79.39442,Harbour Nails,4f2462e0e4b0c41c5296410a,43.636797,-79.400204,Nail Salon
962,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,43.628947,-79.39442,Beauty Line Nail Spa,4c79484897028cfaa37adafe,43.643904,-79.402039,Nail Salon
963,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,43.628947,-79.39442,Sculpture Nails and Spa,4f931759e4b03c10533083bb,43.638168,-79.390608,Nail Salon
964,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,43.628947,-79.39442,Organic Nail Bar,577d8130498eeb45a9049858,43.638389,-79.400691,Nail Salon


In [13]:
target = "Church and Wellesley"

# Filter venues in the target neighborhood
search_area_M4Y = toronto_venues[toronto_venues['Neighborhood'].str.contains(target, na=False)]
latitude = toronto_DF[toronto_DF['Neighbourhood'].str.contains(target, na=False)]['Latitude'].iloc[0]
longitude = toronto_DF[toronto_DF['Neighbourhood'].str.contains(target, na=False)]['Longitude'].iloc[0]

In [14]:
target = "Brockton, Parkdale Village, Exhibition Place"

# Filter venues in the target neighborhood
search_area_M6K = toronto_venues[toronto_venues['Neighborhood'].str.contains(target, na=False)]
latitude = toronto_DF[toronto_DF['Neighbourhood'].str.contains(target, na=False)]['Latitude'].iloc[0]
longitude = toronto_DF[toronto_DF['Neighbourhood'].str.contains(target, na=False)]['Longitude'].iloc[0]

In [15]:
target = "Willowdale South"

# Filter venues in the target neighborhood
search_area_M2N = toronto_venues[toronto_venues['Neighborhood'].str.contains(target, na=False)]
latitude = toronto_DF[toronto_DF['Neighbourhood'].str.contains(target, na=False)]['Latitude'].iloc[0]
longitude = toronto_DF[toronto_DF['Neighbourhood'].str.contains(target, na=False)]['Longitude'].iloc[0]

In [16]:
target = "Regent Park, Harbourfront"

# Filter venues in the target neighborhood
search_area_M5A = toronto_venues[toronto_venues['Neighborhood'].str.contains(target, na=False)]
latitude = toronto_DF[toronto_DF['Neighbourhood'].str.contains(target, na=False)]['Latitude'].iloc[0]
longitude = toronto_DF[toronto_DF['Neighbourhood'].str.contains(target, na=False)]['Longitude'].iloc[0]

In [17]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, venue, neighborhood in zip(search_area_M5V['Venue Latitude'], search_area_M5V['Venue Longitude'], search_area_M5V['Venue'], search_area_M5V['Neighborhood']):
    label = '{},{}'.format(venue, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

In [18]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, venue, neighborhood in zip(search_area_M4Y['Venue Latitude'], search_area_M4Y['Venue Longitude'], search_area_M4Y['Venue'], search_area_M4Y['Neighborhood']):
    label = '{},{}'.format(venue, neighborhood)
    label = folium.Popup4y(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [19]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, venue, neighborhood in zip(search_area_M6K['Venue Latitude'], search_area_M6K['Venue Longitude'], search_area_M6K['Venue'], search_area_M6K['Neighborhood']):
    label = '{},{}'.format(venue, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [20]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, venue, neighborhood in zip(search_area_M2N['Venue Latitude'], search_area_M2N['Venue Longitude'], search_area_M2N['Venue'], search_area_M2N['Neighborhood']):
    label = '{},{}'.format(venue, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [21]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, venue, neighborhood in zip(search_area_M5A['Venue Latitude'], search_area_M5A['Venue Longitude'], search_area_M5A['Venue'], search_area_M5A['Neighborhood']):
    label = '{},{}'.format(venue, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

### Part 3. Customer Reviews
It seems like there is one one pet groomer withing 2km of Scarborough Village, I am interested in seeing customer's comments on Funny Bunny.  I can use [Foursquare's Place Tips API.](https://docs.foursquare.com/developer/reference/place-tips)


In [22]:
# a function to loop through the list of nail stores in the neighborhood and compile all the comments related to them
def getTips(venues, fsq_ids):
    venues_tips = []

    for venue, fsq_id in zip(venues, fsq_ids):
        print(f"Processing venue: {venue}")


        # Create the API request URL and parameters
        url = f"https://api.foursquare.com/v3/places/{fsq_id}/tips"

        # Set up headers with the API key
        headers = {
            "Accept": "application/json",
            "Authorization": API_KEY  
        }
        
        # Make the GET request
        response = requests.get(url, headers=headers)

        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to get data for {venue}. Status code: {response.status_code}")
            print(f"Error message: {response.text}")
            continue

        results = response.json()
        
        # Parse the JSON response
        for tip in results:
            # Extract venue information
            tip_id = tip.get('id')
            tip_created = tip.get('created_at')
            tip_text = tip.get('text')
   

            # Append to the list
            venues_tips.append([
                venues,
                tip_id,
                tip_created,
                tip_text
            ])
            
    
    # Create a DataFrame from the list
    nearby_tips = pd.DataFrame(venues_tips, columns=[
        'venue_name',
        'id',
        'created_at',
        'text'
    ])

    return nearby_tips


nearby_tips = getTips(
    venues=search_area_M5V['Venue'],
    fsq_ids=search_area_M5V['fsq_id']
)

# Display the first few rows of the resulting DataFrame
nearby_tips.head()

Processing venue: Yeti Nails & Spa
Processing venue: Pinky's Nails
Processing venue: Sparks
Processing venue: The Ten Spot
Processing venue: Polish'd Nail Bar
Processing venue: Rite Spa
Processing venue: Harbour Nails
Processing venue: Beauty Line Nail Spa
Processing venue: Sculpture Nails and Spa
Processing venue: Organic Nail Bar
Processing venue: Harbour Nails 2
Processing venue: Nuvo Nails
Processing venue: She Flirts
Processing venue: Nice One Nails
Processing venue: La Nails
Processing venue: Lydia Nails
Processing venue: Sparks Salons
Processing venue: Naked Beauty Bar
Processing venue: Organic Nail Bar
Processing venue: Luxe Beauty


,venue_name,id,created_at,text
0,955 Yeti Nails & Spa 956 ...,5522be70498e143bdeef29ea,2015-04-06T17:12:16.000Z,Yeti Nails & Spa is awesome! They make the exp...
1,955 Yeti Nails & Spa 956 ...,582c1e45d9155a3a5284311e,2016-11-16T08:52:21.000Z,This is by far the best manicure I’ve ever had...
2,955 Yeti Nails & Spa 956 ...,5ac5fae095d986020973e3c9,2018-04-05T10:30:56.000Z,"The water circuit was unique (steam, salt wate..."
3,955 Yeti Nails & Spa 956 ...,5a1bf2ea25ecca6267ce2c26,2017-11-27T11:11:38.000Z,I have been going to YETI NAILS & SPA for year...
4,955 Yeti Nails & Spa 956 ...,59ef1eb7947c0529fe908dce,2017-10-24T11:06:31.000Z,"Great salon, really friendly. Had the hot wax ..."


In [23]:
nearby_tips_M4Y = getTips(
    venues=search_area_M4Y['Venue'],
    fsq_ids=search_area_M4Y['fsq_id']
)

# Display the first few rows of the resulting DataFrame
nearby_tips.head()

Processing venue: Yeti Nails & Spa
Processing venue: Mani Pedi Spa
Processing venue: Bubble Bath Spa & Nails
Processing venue: Urban Nails
Processing venue: M Nails
Processing venue: Pantages Suites Hotel & Spa
Processing venue: Shining Nails and Spa
Processing venue: Neisha's Nails & Esthetics
Processing venue: The Cube Nail Lab
Processing venue: M Lovely Nails
Processing venue: C Nails
Processing venue: Queen's Nails
Processing venue: Grace Nails Salon
Processing venue: Nara's Mani & Pedi Spa
Processing venue: Polish'd Nail Bar
Processing venue: Nails on Yonge
Processing venue: Le Blanc Nails Bar
Processing venue: Sun Nails
Processing venue: Melissa Nails
Processing venue: Celebrity Nail Studio


,venue_name,id,created_at,text
0,955 Yeti Nails & Spa 956 ...,5522be70498e143bdeef29ea,2015-04-06T17:12:16.000Z,Yeti Nails & Spa is awesome! They make the exp...
1,955 Yeti Nails & Spa 956 ...,582c1e45d9155a3a5284311e,2016-11-16T08:52:21.000Z,This is by far the best manicure I’ve ever had...
2,955 Yeti Nails & Spa 956 ...,5ac5fae095d986020973e3c9,2018-04-05T10:30:56.000Z,"The water circuit was unique (steam, salt wate..."
3,955 Yeti Nails & Spa 956 ...,5a1bf2ea25ecca6267ce2c26,2017-11-27T11:11:38.000Z,I have been going to YETI NAILS & SPA for year...
4,955 Yeti Nails & Spa 956 ...,59ef1eb7947c0529fe908dce,2017-10-24T11:06:31.000Z,"Great salon, really friendly. Had the hot wax ..."


In [24]:
nearby_tips_M6K = getTips(
    venues=search_area_M6K['Venue'],
    fsq_ids=search_area_M6K['fsq_id']
)

# Display the first few rows of the resulting DataFrame
nearby_tips.head()

Processing venue: Sparks
Processing venue: Pinky's Nails
Processing venue: The Ten Spot
Processing venue: Kim Nails & Spa
Processing venue: X Nails & Spa Ltd
Processing venue: Queen Nails & Spa
Processing venue: Cures Studio
Processing venue: Luxe Beauty
Processing venue: Sparks Salons
Processing venue: Lydia Nails
Processing venue: Organic Nail Bar
Processing venue: Buffer Nails and Waxing Inc
Processing venue: Beauty Line Nail Spa
Processing venue: Tips Nail Bar
Processing venue: La Nails
Processing venue: La Belle Nails and Spa
Processing venue: Show Nails Salon
Processing venue: Sekai Nail & Beauty Bar
Processing venue: Soho Spa
Processing venue: Naked Beauty Bar


,venue_name,id,created_at,text
0,955 Yeti Nails & Spa 956 ...,5522be70498e143bdeef29ea,2015-04-06T17:12:16.000Z,Yeti Nails & Spa is awesome! They make the exp...
1,955 Yeti Nails & Spa 956 ...,582c1e45d9155a3a5284311e,2016-11-16T08:52:21.000Z,This is by far the best manicure I’ve ever had...
2,955 Yeti Nails & Spa 956 ...,5ac5fae095d986020973e3c9,2018-04-05T10:30:56.000Z,"The water circuit was unique (steam, salt wate..."
3,955 Yeti Nails & Spa 956 ...,5a1bf2ea25ecca6267ce2c26,2017-11-27T11:11:38.000Z,I have been going to YETI NAILS & SPA for year...
4,955 Yeti Nails & Spa 956 ...,59ef1eb7947c0529fe908dce,2017-10-24T11:06:31.000Z,"Great salon, really friendly. Had the hot wax ..."


In [25]:
nearby_tips_M2N = getTips(
    venues=search_area_M2N['Venue'],
    fsq_ids=search_area_M2N['fsq_id']
)

# Display the first few rows of the resulting DataFrame
nearby_tips.head()

Processing venue: Nail Lab
Processing venue: Lux Nail Bar
Processing venue: Plush by Marina
Processing venue: Beauty Secrets Spa Boutique
Processing venue: Bunnynails
Processing venue: Polish Me Parlour
Processing venue: Mansoore Hair Salon
Processing venue: Le Code Nail Bar
Processing venue: Nail For You
Processing venue: Lisa's Nails Spa
Processing venue: Daisy Nails & Spa
Processing venue: Breezy Nails & Spa
Processing venue: Splendid Nails Ltd
Processing venue: Emi Nails
Processing venue: Clover Nails Spa
Processing venue: Dazzle Nail Spa
Processing venue: Gorgeous Nails & Spa
Processing venue: Fresh Island Nail Salon
Processing venue: Nails Republic
Processing venue: CoCo Nail Bar


,venue_name,id,created_at,text
0,955 Yeti Nails & Spa 956 ...,5522be70498e143bdeef29ea,2015-04-06T17:12:16.000Z,Yeti Nails & Spa is awesome! They make the exp...
1,955 Yeti Nails & Spa 956 ...,582c1e45d9155a3a5284311e,2016-11-16T08:52:21.000Z,This is by far the best manicure I’ve ever had...
2,955 Yeti Nails & Spa 956 ...,5ac5fae095d986020973e3c9,2018-04-05T10:30:56.000Z,"The water circuit was unique (steam, salt wate..."
3,955 Yeti Nails & Spa 956 ...,5a1bf2ea25ecca6267ce2c26,2017-11-27T11:11:38.000Z,I have been going to YETI NAILS & SPA for year...
4,955 Yeti Nails & Spa 956 ...,59ef1eb7947c0529fe908dce,2017-10-24T11:06:31.000Z,"Great salon, really friendly. Had the hot wax ..."


In [26]:
nearby_tips_M5A = getTips(
    venues=search_area_M5A['Venue'],
    fsq_ids=search_area_M5A['fsq_id']
)

# Display the first few rows of the resulting DataFrame
nearby_tips.head()

Processing venue: Urban Nails
Processing venue: Bubble Bath Spa & Nails
Processing venue: M Nails
Processing venue: Star Nails
Processing venue: Pantages Suites Hotel & Spa
Processing venue: Polish'd Nail Bar
Processing venue: Nailing
Processing venue: Rainbow Nails & Spa
Processing venue: 6ix Tips Nails & Spa
Processing venue: Canada Nails
Processing venue: Luminous Nail
Processing venue: Star Nails
Processing venue: Precision Nail Fungus Removal
Processing venue: Tip Top Nails
Processing venue: Tweetie Nails & Spa
Processing venue: Dragon Nails
Processing venue: Vanna Classic Spa
Processing venue: Poli Spa
Processing venue: Jessi Nails
Processing venue: Fashion Nails Beauty Spa


,venue_name,id,created_at,text
0,955 Yeti Nails & Spa 956 ...,5522be70498e143bdeef29ea,2015-04-06T17:12:16.000Z,Yeti Nails & Spa is awesome! They make the exp...
1,955 Yeti Nails & Spa 956 ...,582c1e45d9155a3a5284311e,2016-11-16T08:52:21.000Z,This is by far the best manicure I’ve ever had...
2,955 Yeti Nails & Spa 956 ...,5ac5fae095d986020973e3c9,2018-04-05T10:30:56.000Z,"The water circuit was unique (steam, salt wate..."
3,955 Yeti Nails & Spa 956 ...,5a1bf2ea25ecca6267ce2c26,2017-11-27T11:11:38.000Z,I have been going to YETI NAILS & SPA for year...
4,955 Yeti Nails & Spa 956 ...,59ef1eb7947c0529fe908dce,2017-10-24T11:06:31.000Z,"Great salon, really friendly. Had the hot wax ..."


### Now it is your turn to discover interesting search results using Foursquare
You are expected to read through the API documentation and customize the searches.  

In [27]:
def getVenueDetails(venues, fsq_ids):
    """
    Simplified version to only get name, website, rating and stats
    """
    venues_details = []
    
    for venue, fsq_id in zip(venues, fsq_ids):
        print(f"Processing venue details: {venue}")
        
        url = f"https://api.foursquare.com/v3/places/{fsq_id}"
        
        # Only request the specific fields we need
        params = {
            'fields': 'name,website,rating,stats'
        }
        
        headers = {
            "Accept": "application/json",
            "Authorization": API_KEY
        }
        
        try:
            response = requests.get(url, headers=headers, params=params)
            if response.status_code == 200:
                data = response.json()
                
                # Extract only the fields we need
                venue_detail = {
                    'venue_name': venue,
                    'fsq_id': fsq_id,
                    'rating': data.get('rating', None),
                    'website': data.get('website', None)
                }
                
                # Get stats
                stats = data.get('stats', {})
                if stats:
                    venue_detail['total_ratings'] = stats.get('total_ratings', 0)
                    venue_detail['total_tips'] = stats.get('total_tips', 0)
                
                venues_details.append(venue_detail)
                
        except Exception as e:
            print(f"Error processing {venue}: {str(e)}")
            continue
    
    # Create DataFrame
    df = pd.DataFrame(venues_details)
    return df

In [28]:
venue_details = getVenueDetails(
    venues=search_area_M5V['Venue'],
    fsq_ids=search_area_M5V['fsq_id']
)

Processing venue details: Yeti Nails & Spa
Processing venue details: Pinky's Nails
Processing venue details: Sparks
Processing venue details: The Ten Spot
Processing venue details: Polish'd Nail Bar
Processing venue details: Rite Spa
Processing venue details: Harbour Nails
Processing venue details: Beauty Line Nail Spa
Processing venue details: Sculpture Nails and Spa
Processing venue details: Organic Nail Bar
Processing venue details: Harbour Nails 2
Processing venue details: Nuvo Nails
Processing venue details: She Flirts
Processing venue details: Nice One Nails
Processing venue details: La Nails
Processing venue details: Lydia Nails
Processing venue details: Sparks Salons
Processing venue details: Naked Beauty Bar
Processing venue details: Organic Nail Bar
Processing venue details: Luxe Beauty


In [29]:
venue_details_M4Y = getVenueDetails(
    venues=search_area_M4Y['Venue'],
    fsq_ids=search_area_M4Y['fsq_id']
)

Processing venue details: Yeti Nails & Spa
Processing venue details: Mani Pedi Spa
Processing venue details: Bubble Bath Spa & Nails
Processing venue details: Urban Nails
Processing venue details: M Nails
Processing venue details: Pantages Suites Hotel & Spa
Processing venue details: Shining Nails and Spa
Processing venue details: Neisha's Nails & Esthetics
Processing venue details: The Cube Nail Lab
Processing venue details: M Lovely Nails
Processing venue details: C Nails
Processing venue details: Queen's Nails
Processing venue details: Grace Nails Salon
Processing venue details: Nara's Mani & Pedi Spa
Processing venue details: Polish'd Nail Bar
Processing venue details: Nails on Yonge
Processing venue details: Le Blanc Nails Bar
Processing venue details: Sun Nails
Processing venue details: Melissa Nails
Processing venue details: Celebrity Nail Studio


In [30]:
venue_details_M6K = getVenueDetails(
    venues=search_area_M6K['Venue'],
    fsq_ids=search_area_M6K['fsq_id']
)

Processing venue details: Sparks
Processing venue details: Pinky's Nails
Processing venue details: The Ten Spot
Processing venue details: Kim Nails & Spa
Processing venue details: X Nails & Spa Ltd
Processing venue details: Queen Nails & Spa
Processing venue details: Cures Studio
Processing venue details: Luxe Beauty
Processing venue details: Sparks Salons
Processing venue details: Lydia Nails
Processing venue details: Organic Nail Bar
Processing venue details: Buffer Nails and Waxing Inc
Processing venue details: Beauty Line Nail Spa
Processing venue details: Tips Nail Bar
Processing venue details: La Nails
Processing venue details: La Belle Nails and Spa
Processing venue details: Show Nails Salon
Processing venue details: Sekai Nail & Beauty Bar
Processing venue details: Soho Spa
Processing venue details: Naked Beauty Bar


In [31]:
venue_details_M2N = getVenueDetails(
    venues=search_area_M2N['Venue'],
    fsq_ids=search_area_M2N['fsq_id']
)

Processing venue details: Nail Lab
Processing venue details: Lux Nail Bar
Processing venue details: Plush by Marina
Processing venue details: Beauty Secrets Spa Boutique
Processing venue details: Bunnynails
Processing venue details: Polish Me Parlour
Processing venue details: Mansoore Hair Salon
Processing venue details: Le Code Nail Bar
Processing venue details: Nail For You
Processing venue details: Lisa's Nails Spa
Processing venue details: Daisy Nails & Spa
Processing venue details: Breezy Nails & Spa
Processing venue details: Splendid Nails Ltd
Processing venue details: Emi Nails
Processing venue details: Clover Nails Spa
Processing venue details: Dazzle Nail Spa
Processing venue details: Gorgeous Nails & Spa
Processing venue details: Fresh Island Nail Salon
Processing venue details: Nails Republic
Processing venue details: CoCo Nail Bar


In [32]:
venue_details_M5A = getVenueDetails(
    venues=search_area_M5A['Venue'],
    fsq_ids=search_area_M5A['fsq_id']
)

Processing venue details: Urban Nails
Processing venue details: Bubble Bath Spa & Nails
Processing venue details: M Nails
Processing venue details: Star Nails
Processing venue details: Pantages Suites Hotel & Spa
Processing venue details: Polish'd Nail Bar
Processing venue details: Nailing
Processing venue details: Rainbow Nails & Spa
Processing venue details: 6ix Tips Nails & Spa
Processing venue details: Canada Nails
Processing venue details: Luminous Nail
Processing venue details: Star Nails
Processing venue details: Precision Nail Fungus Removal
Processing venue details: Tip Top Nails
Processing venue details: Tweetie Nails & Spa
Processing venue details: Dragon Nails
Processing venue details: Vanna Classic Spa
Processing venue details: Poli Spa
Processing venue details: Jessi Nails
Processing venue details: Fashion Nails Beauty Spa


In [33]:
venue_details

,venue_name,fsq_id,rating,website,total_ratings,total_tips
0,Yeti Nails & Spa,4cc99fdd786e4688d2379509,9.2,http://www.yetinailsnspa.com,296.0,213.0
1,Pinky's Nails,4fe254576d86fad7b5763fc6,8.1,http://www.pinkysnailsto.com,11.0,0.0
2,Sparks,4d5596f3cff7721e788eabf5,8.1,http://www.sparkssalons.com,11.0,5.0
3,The Ten Spot,4aee01bef964a52034d121e3,7.7,http://www.the10spot.com,38.0,15.0
4,Polish'd Nail Bar,5077398de4b0eb9f2a41e7ec,6.1,http://polishdnailbar.com,10.0,3.0
5,Rite Spa,59ac224c3c858d09e3563900,NaN,None,NaN,NaN
6,Harbour Nails,4f2462e0e4b0c41c5296410a,NaN,None,NaN,NaN
7,Beauty Line Nail Spa,4c79484897028cfaa37adafe,5.9,None,8.0,6.0
8,Sculpture Nails and Spa,4f931759e4b03c10533083bb,NaN,http://www.biosculpturenails.com/en,NaN,NaN
9,Organic Nail Bar,577d8130498eeb45a9049858,NaN,https://organicnailbar.ca,NaN,NaN


In [40]:
M5V = pd.merge(search_area_M5V,venue_details,how="outer",on="fsq_id") 
M5V_droped=M5V.dropna()
M5V_droped

,Neighborhood,Postalcode,Neighborhood Latitude,Neighborhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category,venue_name,rating,website,total_ratings,total_tips
0,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,43.628947,-79.39442,Yeti Nails & Spa,4cc99fdd786e4688d2379509,43.647996,-79.396708,Nail Salon,Yeti Nails & Spa,9.2,http://www.yetinailsnspa.com,296.0,213.0
1,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,43.628947,-79.39442,Pinky's Nails,4fe254576d86fad7b5763fc6,43.646036,-79.406679,Nail Salon,Pinky's Nails,8.1,http://www.pinkysnailsto.com,11.0,0.0
2,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,43.628947,-79.39442,Sparks,4d5596f3cff7721e788eabf5,43.641710,-79.414449,Nail Salon,Sparks,8.1,http://www.sparkssalons.com,11.0,5.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,43.628947,-79.39442,The Ten Spot,4aee01bef964a52034d121e3,43.646453,-79.407120,Nail Salon,The Ten Spot,7.7,http://www.the10spot.com,38.0,15.0
4,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,43.628947,-79.39442,Polish'd Nail Bar,5077398de4b0eb9f2a41e7ec,43.648120,-79.383405,Nail Salon,Polish'd Nail Bar,6.1,http://polishdnailbar.com,10.0,3.0


In [35]:
M6K = pd.merge(search_area_M6K,venue_details_M6K,how="outer",on="fsq_id") 
M6K_droped=M6K.dropna()
M6K_droped

,Neighborhood,Postalcode,Neighborhood Latitude,Neighborhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category,venue_name,rating,website,total_ratings,total_tips
0,"Brockton, Parkdale Village, Exhibition Place",M6K,43.636847,-79.428191,Sparks,4d5596f3cff7721e788eabf5,43.641710,-79.414449,Nail Salon,Sparks,8.1,http://www.sparkssalons.com,11.0,5.0
1,"Brockton, Parkdale Village, Exhibition Place",M6K,43.636847,-79.428191,Pinky's Nails,4fe254576d86fad7b5763fc6,43.646036,-79.406679,Nail Salon,Pinky's Nails,8.1,http://www.pinkysnailsto.com,11.0,0.0
2,"Brockton, Parkdale Village, Exhibition Place",M6K,43.636847,-79.428191,The Ten Spot,4aee01bef964a52034d121e3,43.646453,-79.407120,Nail Salon,The Ten Spot,7.7,http://www.the10spot.com,38.0,15.0


In [36]:
M4Y = pd.merge(search_area_M4Y,venue_details_M4Y,how="outer",on="fsq_id") 
M4Y_droped=M4Y.dropna()
M4Y_droped

,Neighborhood,Postalcode,Neighborhood Latitude,Neighborhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category,venue_name,rating,website,total_ratings,total_tips
0,Church and Wellesley,M4Y,43.66586,-79.38316,Yeti Nails & Spa,4cc99fdd786e4688d2379509,43.647996,-79.396708,Nail Salon,Yeti Nails & Spa,9.2,http://www.yetinailsnspa.com,296.0,213.0
1,Church and Wellesley,M4Y,43.66586,-79.38316,Mani Pedi Spa,4bca0140cc8cd13a00acbccf,43.682726,-79.391583,Nail Salon,Mani Pedi Spa,8.5,http://manipedispa.ca,22.0,8.0
2,Church and Wellesley,M4Y,43.66586,-79.38316,Bubble Bath Spa & Nails,512ce732e4b0aa0ac5f50f05,43.659368,-79.385430,Nail Salon,Bubble Bath Spa & Nails,7.6,http://www.scottmission.com,10.0,2.0
3,Church and Wellesley,M4Y,43.66586,-79.38316,Urban Nails,4ae4bc2ff964a520909d21e3,43.676453,-79.356485,Nail Salon,Urban Nails,8.3,http://www.urbannails.ca,38.0,17.0
4,Church and Wellesley,M4Y,43.66586,-79.38316,M Nails,4bb66377f562ef3b53143097,43.676176,-79.357984,Nail Salon,M Nails,7.6,http://www.mnails.ca,10.0,8.0
5,Church and Wellesley,M4Y,43.66586,-79.38316,Pantages Suites Hotel & Spa,4ae61cf6f964a520caa421e3,43.654554,-79.379053,Nail Salon,Pantages Suites Hotel & Spa,6.9,http://www.pantageshotel.com,53.0,17.0
14,Church and Wellesley,M4Y,43.66586,-79.38316,Polish'd Nail Bar,5077398de4b0eb9f2a41e7ec,43.648120,-79.383405,Nail Salon,Polish'd Nail Bar,6.1,http://polishdnailbar.com,10.0,3.0


In [37]:
M2N = pd.merge(search_area_M2N,venue_details_M2N,how="outer",on="fsq_id") 
M2N_droped=M2N.dropna()
M2N_droped

,Neighborhood,Postalcode,Neighborhood Latitude,Neighborhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category,venue_name,rating,website,total_ratings,total_tips
1,Willowdale South,M2N,43.77012,-79.408493,Lux Nail Bar,558dcb68498ed8eb8ddf1f4a,43.764519,-79.396204,Nail Salon,Lux Nail Bar,5.9,http://luxnailbar.ca,8.0,1.0


In [45]:
M5A = pd.merge(search_area_M5A,venue_details_M5A,how="outer",on="fsq_id") 
M5A_droped=M5A.dropna()
M5A_droped

,Neighborhood,Postalcode,Neighborhood Latitude,Neighborhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category,venue_name,rating,website,total_ratings,total_tips
0,"Regent Park, Harbourfront",M5A,43.65426,-79.360636,Urban Nails,4ae4bc2ff964a520909d21e3,43.676453,-79.356485,Nail Salon,Urban Nails,8.3,http://www.urbannails.ca,38.0,17.0
1,"Regent Park, Harbourfront",M5A,43.65426,-79.360636,Bubble Bath Spa & Nails,512ce732e4b0aa0ac5f50f05,43.659368,-79.385430,Nail Salon,Bubble Bath Spa & Nails,7.6,http://www.scottmission.com,10.0,2.0
2,"Regent Park, Harbourfront",M5A,43.65426,-79.360636,M Nails,4bb66377f562ef3b53143097,43.676176,-79.357984,Nail Salon,M Nails,7.6,http://www.mnails.ca,10.0,8.0
3,"Regent Park, Harbourfront",M5A,43.65426,-79.360636,Star Nails,4afae339f964a520581922e3,43.668385,-79.340708,Nail Salon,Star Nails,7.3,http://www.starnails.ca,12.0,7.0
4,"Regent Park, Harbourfront",M5A,43.65426,-79.360636,Pantages Suites Hotel & Spa,4ae61cf6f964a520caa421e3,43.654554,-79.379053,Nail Salon,Pantages Suites Hotel & Spa,6.9,http://www.pantageshotel.com,53.0,17.0
5,"Regent Park, Harbourfront",M5A,43.65426,-79.360636,Polish'd Nail Bar,5077398de4b0eb9f2a41e7ec,43.648120,-79.383405,Nail Salon,Polish'd Nail Bar,6.1,http://polishdnailbar.com,10.0,3.0


In [46]:
# After getting the dataframe for all the potential neighbourhood, we can concatenate them into one dataframe
PotentialNeighbourhood = pd.concat([M5V_droped,M4Y_droped,M6K_droped,M2N_droped,M5A_droped]) 
PotentialNeighbourhood

,Neighborhood,Postalcode,Neighborhood Latitude,Neighborhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category,venue_name,rating,website,total_ratings,total_tips
0,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,43.628947,-79.394420,Yeti Nails & Spa,4cc99fdd786e4688d2379509,43.647996,-79.396708,Nail Salon,Yeti Nails & Spa,9.2,http://www.yetinailsnspa.com,296.0,213.0
1,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,43.628947,-79.394420,Pinky's Nails,4fe254576d86fad7b5763fc6,43.646036,-79.406679,Nail Salon,Pinky's Nails,8.1,http://www.pinkysnailsto.com,11.0,0.0
2,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,43.628947,-79.394420,Sparks,4d5596f3cff7721e788eabf5,43.641710,-79.414449,Nail Salon,Sparks,8.1,http://www.sparkssalons.com,11.0,5.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,43.628947,-79.394420,The Ten Spot,4aee01bef964a52034d121e3,43.646453,-79.407120,Nail Salon,The Ten Spot,7.7,http://www.the10spot.com,38.0,15.0
4,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,43.628947,-79.394420,Polish'd Nail Bar,5077398de4b0eb9f2a41e7ec,43.648120,-79.383405,Nail Salon,Polish'd Nail Bar,6.1,http://polishdnailbar.com,10.0,3.0
0,Church and Wellesley,M4Y,43.665860,-79.383160,Yeti Nails & Spa,4cc99fdd786e4688d2379509,43.647996,-79.396708,Nail Salon,Yeti Nails & Spa,9.2,http://www.yetinailsnspa.com,296.0,213.0
1,Church and Wellesley,M4Y,43.665860,-79.383160,Mani Pedi Spa,4bca0140cc8cd13a00acbccf,43.682726,-79.391583,Nail Salon,Mani Pedi Spa,8.5,http://manipedispa.ca,22.0,8.0
2,Church and Wellesley,M4Y,43.665860,-79.383160,Bubble Bath Spa & Nails,512ce732e4b0aa0ac5f50f05,43.659368,-79.385430,Nail Salon,Bubble Bath Spa & Nails,7.6,http://www.scottmission.com,10.0,2.0
3,Church and Wellesley,M4Y,43.665860,-79.383160,Urban Nails,4ae4bc2ff964a520909d21e3,43.676453,-79.356485,Nail Salon,Urban Nails,8.3,http://www.urbannails.ca,38.0,17.0
4,Church and Wellesley,M4Y,43.665860,-79.383160,M Nails,4bb66377f562ef3b53143097,43.676176,-79.357984,Nail Salon,M Nails,7.6,http://www.mnails.ca,10.0,8.0


In [47]:
PotentialNeighbourhood.to_csv("PotentialNeighborhood.csv", index=False)